In [4]:
!python -m pip install mxnet
!python -m pip install -U d2l

import os
from mxnet import np, npx
from d2l import mxnet as d2l

npx.set_np()

     |████████████████████████████████| 77 kB 4.0 MB/s eta 0:00:011


In [5]:
#@save
d2l.DATA_HUB['glove.6b.50d'] = (d2l.DATA_URL + 'glove.6B.50d.zip',
                                '0b8703943ccdb6eb788e6f091b8946e82231bc4d')

#@save
d2l.DATA_HUB['glove.6b.100d'] = (d2l.DATA_URL + 'glove.6B.100d.zip',
                                 'cd43bfb07e44e6f27cbcc7bc9ae3d80284fdaf5a')

#@save
d2l.DATA_HUB['glove.42b.300d'] = (d2l.DATA_URL + 'glove.42B.300d.zip',
                                  'b5116e234e9eb9076672cfeabf5469f3eec904fa')

#@save
d2l.DATA_HUB['wiki.en'] = (d2l.DATA_URL + 'wiki.en.zip',
                           'c1816da3821ae9f43899be655002f6c723e91b88')

In [6]:
#@save
class TokenEmbedding:
    """Token Embedding."""
    def __init__(self, embedding_name):
        self.idx_to_token, self.idx_to_vec = self._load_embedding(
            embedding_name)
        self.unknown_idx = 0
        self.token_to_idx = {
            token: idx for idx, token in enumerate(self.idx_to_token)}

    def _load_embedding(self, embedding_name):
        idx_to_token, idx_to_vec = ['<unk>'], []
        data_dir = d2l.download_extract(embedding_name)
        # GloVe website: https://nlp.stanford.edu/projects/glove/
        # fastText website: https://fasttext.cc/
        with open(os.path.join(data_dir, 'vec.txt'), 'r') as f:
            for line in f:
                elems = line.rstrip().split(' ')
                token, elems = elems[0], [float(elem) for elem in elems[1:]]
                # Skip header information, such as the top row in fastText
                if len(elems) > 1:
                    idx_to_token.append(token)
                    idx_to_vec.append(elems)
        idx_to_vec = [[0] * len(idx_to_vec[0])] + idx_to_vec
        return idx_to_token, np.array(idx_to_vec)

    def __getitem__(self, tokens):
        indices = [
            self.token_to_idx.get(token, self.unknown_idx)
            for token in tokens]
        vecs = self.idx_to_vec[np.array(indices)]
        return vecs

    def __len__(self):
        return len(self.idx_to_token)

In [7]:
glove_6b50d = TokenEmbedding('glove.6b.50d')


In [8]:
len(glove_6b50d)


400001

In [9]:
glove_6b50d.token_to_idx['beautiful'], glove_6b50d.idx_to_token[3367]


(3367, 'beautiful')

In [10]:
def knn(W, x, k):
    # The added 1e-9 is for numerical stability
    cos = np.dot(W, x.reshape(
        -1,)) / (np.sqrt(np.sum(W * W, axis=1) + 1e-9) * np.sqrt(
            (x * x).sum()))
    topk = npx.topk(cos, k=k, ret_typ='indices')
    return topk, [cos[int(i)] for i in topk]

In [11]:
def get_similar_tokens(query_token, k, embed):
    topk, cos = knn(embed.idx_to_vec, embed[[query_token]], k + 1)
    for i, c in zip(topk[1:], cos[1:]):  # Remove input words
        print(f'cosine sim={float(c):.3f}: {embed.idx_to_token[int(i)]}')

In [15]:
get_similar_tokens('chip', 3, glove_6b50d)

cosine sim=0.856: chips
cosine sim=0.749: intel
cosine sim=0.749: electronics


In [13]:
get_similar_tokens('baby', 3, glove_6b50d)

cosine sim=0.839: babies
cosine sim=0.800: boy
cosine sim=0.792: girl


In [14]:
get_similar_tokens('beautiful', 3, glove_6b50d)

cosine sim=0.921: lovely
cosine sim=0.893: gorgeous
cosine sim=0.830: wonderful


In [22]:
get_similar_tokens('hypothesizes', 20, glove_6b50d)

cosine sim=0.847: surmises
cosine sim=0.832: theorises
cosine sim=0.829: theorizes
cosine sim=0.825: infers
cosine sim=0.815: theorised
cosine sim=0.806: posited
cosine sim=0.780: opines
cosine sim=0.780: supposes
cosine sim=0.775: hypothesised
cosine sim=0.769: speculates
cosine sim=0.768: ascertains
cosine sim=0.767: inferring
cosine sim=0.764: posits
cosine sim=0.751: discerns
cosine sim=0.749: averred
cosine sim=0.748: positing
cosine sim=0.748: postulating
cosine sim=0.740: surmising
cosine sim=0.738: hypothesizing
cosine sim=0.731: intimating


In [32]:
get_similar_tokens('discuss', 20, glove_6b50d)

cosine sim=0.923: discussed
cosine sim=0.892: talks
cosine sim=0.891: discussions
cosine sim=0.876: discussing
cosine sim=0.864: meeting
cosine sim=0.828: consultations
cosine sim=0.827: negotiations
cosine sim=0.824: meet
cosine sim=0.820: cooperation
cosine sim=0.816: meetings
cosine sim=0.810: resume
cosine sim=0.798: hoped
cosine sim=0.794: agreed
cosine sim=0.794: discussion
cosine sim=0.792: agreement
cosine sim=0.790: resolve
cosine sim=0.788: stressed
cosine sim=0.782: relations
cosine sim=0.782: agree
cosine sim=0.780: possibility
